In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# 失敗的話會列印出：'沒讀到 HUGGINGFACEHUB_API key'
os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [2]:
from langchain.llms import HuggingFaceEndpoint
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

## Get B/S & I/S Item

In [3]:
import requests
import pandas as pd

In [4]:
@tool
def GetIncomeStatementItem(income_statement_item: str, ticker: int, year: int, season: int) -> float:
    '''
    A function that will get the income statement item of the ticker in the specific year and season.
    Returns -1 if something went wrong or the item is not found.
    
    If the return value is not -1, the result is correct. It's NOT an ID of any sort.
    Please return the income statement item's value and stop searching immediately.
    '''
    
    try:
        req = requests.post("https://mops.twse.com.tw/mops/web/ajax_t164sb04", data={
            "encodeURIComponent": 1,
            "step": 1,
            "firstin": 1,
            "off": 1,
            "queryName": ticker,
            "inpuType": ticker,
            "TYPEK": "all",
            "isnew": "false",
            "co_id": ticker,
            "year": year,
            "season": season
        })

        df = pd.read_html(req.text)[1]
        df = df.iloc[:, :2].dropna()
        display(df)
        for i, rows in df.iterrows():
            if income_statement_item in rows.iloc[0]:  # item found
                return float(rows.iloc[1])
        
        return -1
    except:
        return -1

    

@tool
def GetBalancedSheetItem(balanced_sheet_item: str, ticker: int, year: int, season: int) -> float:
    '''
    A function that will get the balanced item of the ticker in the specific year and season.
    Returns -1 if something went wrong. Please stop searching immediately and ask the user to modify their input.
    
    If the return value is not -1, the result is correct. It's NOT an ID of any sort.
    Please return the balanced sheet item's value and stop searching immediately.
    '''
    
    try:
        req = requests.post("https://mops.twse.com.tw/mops/web/ajax_t164sb03", data={
            "encodeURIComponent": 1,
            "step": 1,
            "firstin": 1,
            "off": 1,
            "queryName": ticker,
            "inpuType": ticker,
            "TYPEK": "all",
            "isnew": "false",
            "co_id": ticker,
            "year": year,
            "season": season
        })

        df = pd.read_html(req.text)[1]
        df = df.iloc[:, :2].dropna()
        display(df)
        for i, rows in df.iterrows():
            if balanced_sheet_item in rows.iloc[0]:  # item found
                return float(rows.iloc[1])
        
        return -1
    except:
        return -1

    
@tool
def GetYoYGrowth(previous_year_value:float, current_year_value:float) -> float:
    '''
    A function that will compute the YoY growth of a specific financial statement item.
    The formula is (current_year_value - previous_year_value) / previous_year_value.
    The functions reterns a floating number, not a percentage.
    The growth can be either negative or positive. The function is error-free.
    '''
    return (current_year_value - previous_year_value) / previous_year_value

In [5]:
tools = [GetIncomeStatementItem, GetBalancedSheetItem, GetYoYGrowth]

## Get Seasonal Revenue

In [6]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.agents.format_scratchpad import format_log_to_str

In [7]:
llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2", max_new_tokens=1024, temperature=0.1)

user_template = """
Find the financial statement item of the ticker in the specific year and season.

financial statement item: {financial_statement_item}
ticker: {ticker}
year: {year}
season: {season}

Try to find the item in the income statement first. If not found, try to find it in the balanced sheet.
DO NOT translate the financial statement item into English!


The input for the tools should be formatted as the following example.
```
"action_input": {{
    "balanced_sheet_item": <financial_statement_item, a string>,
    "ticker": <ticker, an integer>,
    "year": <year, an integer>,
    "season": <season, an integer>
}}
```

Do NOT invoke GetYoYGrowth.
"""

prompt = ChatPromptTemplate.from_messages([
    ("user", user_template),
    ("placeholder", "{agent_scratchpad}")
])

C:\Users\ducky\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\ducky\.cache\huggingface\token
Login successful


In [8]:
agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # supports two argument tools
    verbose = True
)

agent_executor = prompt | agent

C:\Users\ducky\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
result = agent_executor.invoke({"financial_statement_item": "營業收入", "ticker": 3665, "year": 113, "season": 1})

print(result["output"])



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "GetIncomeStatementItem",
  "action_input": {
    "income_statement_item": "營業收入",
    "ticker": 3665,
    "year": 113,
    "season": 1
  }
}
```


民國113年第1季                      
                         單位：新台幣仟元                      
                             會計項目 113年01月01日至113年03月31日
               Unnamed: 0_level_3                    金額
0                          營業收入合計           12425561.00
1                          營業成本合計            9152624.00
2                        營業毛利（毛損）            3272937.00
3                      營業毛利（毛損）淨額            3272937.00
5                            推銷費用             703649.00
6                            管理費用            1129743.00
7                          研究發展費用             346680.00
8                    預期信用減損損失（利益）              -8562.00
9                          營業費用合計            2171510.00
10                       營業利益（損失）            1101427.00
12                           利息收入              96891.00
13                           其他收入              35007.00
14                      其他利益及損失淨額             -74102.00
15                         財務成本淨額             231085.00
16         採用權益法認列之關聯企業及合資損益之份額淨額                 11.00
17                     營業外收入及支出合計            -173278.00
18                       稅前淨利（淨損）             928149.00
19                    所得稅費用（利益）合計             350147.00
20                 繼續營業單位本期淨利（淨損）             578002.00
21                       本期淨利（淨損）             578002.00
23                    確定福利計畫之再衡量數               9890.00
24  透過其他綜合損益按公允價值衡量之權益工具投資未實現評價損益             -12602.00
25                避險工具之損益－不重分類至損益               9905.00
26                   不重分類至損益之其他項目            1016750.00
27                 與不重分類之項目相關之所得稅               3992.00
28                    不重分類至損益之項目：            1019951.00
29              國外營運機構財務報表換算之兌換差額            -618233.00
30                 後續可能重分類至損益之項目：            -618233.00
31                     其他綜合損益（淨額）             401718.00
32                       本期綜合損益總額             979720.00
34                    母公司業主（淨利∕損）             580672.00
35                    非控制權益（淨利∕損）              -2670.00
36                    母公司業主（綜合損益）             981215.00
37                    非控制權益（綜合損益）              -1495.00
39                   繼續營業單位淨利（淨損）                  3.56
40                         基本每股盈餘                  3.56
42                   繼續營業單位淨利（淨損）                  3.44
43                         稀釋每股盈餘                  3.44


Observation: 12425561.0
Thought: I have the income statement item value.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The income statement item value for the financial statement item '營業收入' of ticker 3665 in year 113 and season 1 is 12425561.0."
}
```

> Finished chain.
The income statement item value for the financial statement item '營業收入' of ticker 3665 in year 113 and season 1 is 12425561.0.


## Get YoY Growth

In [11]:
user_template = """
Find the YoY growth rate of the financial statement item of the ticker from year1 to year2.

financial statement item: {financial_statement_item}
ticker: {ticker}
year1: {year1}
year2: {year2}
season: 4

Try to find the item in the income statement first. If not found, try to find it in the balanced sheet.
DO NOT translate the financial statement item into English, and DO NOT change the input years!


The input for the tools should be formatted as the following example.
```
"action_input": {{
    "balanced_sheet_item": <financial_statement_item, a string>,
    "ticker": <ticker, an integer>,
    "year": <year, an integer>,
    "season": <season, an integer>
}}
```

You must get the financial statement item values of the two years before invoking GetYoYGrowth.

Present your final answer as the following example.
```
{{
  "action": "Final Answer",
  "action_input": "The YoY growth rate of the cash item for ticker <ticker> from year <year1> to year <year2> in season 4 is approximately <yoy, in percentage>."
}}
```
"""


prompt = ChatPromptTemplate.from_messages([
    ("user", user_template),
    ("placeholder", "{agent_scratchpad}")
])

agent_executor = prompt | agent

ppe_result = agent_executor.invoke({"financial_statement_item": "不動產、廠房及設備", "ticker": 3665, "year1": 111, "year2": 112})
print(ppe_result["output"])



> Entering new AgentExecutor chain...
Thought: Understood, I will follow the instructions and use the provided tools to find the YoY growth rate of the specified financial statement item for the given ticker and years.

Action:
```json
{
  "action": "GetIncomeStatementItem",
  "action_input": {
    "income_statement_item": "不動產、廠房及設備",
    "ticker": 3665,
    "year": 111,
    "season": 4
  }
}
```



民國111年第4季             
                         單位：新台幣仟元             
                             會計項目        111年度
               Unnamed: 0_level_3           金額
0                          營業收入合計  53757171.00
1                          營業成本合計  39969688.00
2                        營業毛利（毛損）  13787483.00
3                      營業毛利（毛損）淨額  13787483.00
5                            推銷費用   2837188.00
6                            管理費用   3945515.00
7                          研究發展費用   1384227.00
8                    預期信用減損損失（利益）     74078.00
9                          營業費用合計   8241008.00
10                       營業利益（損失）   5546475.00
12                           利息收入     62266.00
13                           其他收入    175906.00
14                      其他利益及損失淨額     33038.00
15                         財務成本淨額    511835.00
16         採用權益法認列之關聯企業及合資損益之份額淨額    -13187.00
17                     營業外收入及支出合計   -253812.00
18                       稅前淨利（淨損）   5292663.00
19                    所得稅費用（利益）合計   1461681.00
20                 繼續營業單位本期淨利（淨損）   3830982.00
21                       本期淨利（淨損）   3830982.00
23                    確定福利計畫之再衡量數    379440.00
24  透過其他綜合損益按公允價值衡量之權益工具投資未實現評價損益    -47233.00
25                避險工具之損益－不重分類至損益    -90350.00
26                   不重分類至損益之其他項目   1953873.00
27                 與不重分類之項目相關之所得稅    109463.00
28                    不重分類至損益之項目：   2086267.00
29              國外營運機構財務報表換算之兌換差額  -1724824.00
30                 後續可能重分類至損益之項目：  -1724824.00
31                     其他綜合損益（淨額）    361443.00
32                       本期綜合損益總額   4192425.00
34                    母公司業主（淨利∕損）   3838380.00
35                    非控制權益（淨利∕損）     -7398.00
36                    母公司業主（綜合損益）   4199000.00
37                    非控制權益（綜合損益）     -6575.00
39                   繼續營業單位淨利（淨損）        25.02
40                         基本每股盈餘        25.02
42                   繼續營業單位淨利（淨損）        23.28
43                         稀釋每股盈餘        23.28


Observation: -1
Thought: The income statement item was not found for the given ticker, year, and season. I will try to find it in the balanced sheet instead.

Action:
```json
{
  "action": "GetBalancedSheetItem",
  "action_input": {
    "balanced_sheet_item": "不動產、廠房及設備",
    "ticker": 3665,
    "year": 111,
    "season": 4
  }
}
```



民國111年第4季            
                     單位：新台幣仟元            
                         會計項目  111年12月31日
           Unnamed: 0_level_3          金額
1                     現金及約當現金   8497568.0
2         透過損益按公允價值衡量之金融資產－流動     23593.0
3            按攤銷後成本衡量之金融資產－流動    393357.0
4                  避險之金融資產－流動      6186.0
5                      應收票據淨額     10854.0
..                        ...         ...
67                       權益總額  22504024.0
68                    負債及權益總計  53446242.0
69              待註銷股本股數（單位：股）         0.0
70    預收股款（權益項下）之約當發行股數（單位：股）         0.0
71  母公司暨子公司所持有之母公司庫藏股股數（單位：股）         0.0

[63 rows x 2 columns]


Observation: 10300936.0
Thought: The balanced sheet item was found for the given ticker, year, and season. I will now find the value for the next year.

Action:
```json
{
  "action": "GetBalancedSheetItem",
  "action_input": {
    "balanced_sheet_item": "不動產、廠房及設備",
    "ticker": 3665,
    "year": 112,
    "season": 4
  }
}
```



民國112年第4季            
                     單位：新台幣仟元            
                         會計項目  112年12月31日
           Unnamed: 0_level_3          金額
1                     現金及約當現金  10627389.0
2         透過損益按公允價值衡量之金融資產－流動     40348.0
3            按攤銷後成本衡量之金融資產－流動   1758561.0
4                  避險之金融資產－流動      3837.0
5                      應收票據淨額    183067.0
..                        ...         ...
73                       權益總額  24597367.0
74                    負債及權益總計  54682928.0
75              待註銷股本股數（單位：股）         0.0
76    預收股款（權益項下）之約當發行股數（單位：股）         0.0
77  母公司暨子公司所持有之母公司庫藏股股數（單位：股）         0.0

[69 rows x 2 columns]


Observation: 12142679.0
Thought: I have the values for the two years. I can now calculate the YoY growth rate.

Action:
```json
{
  "action": "GetYoYGrowth",
  "action_input": {
    "previous_year_value": 10300936.0,
    "current_year_value": 12142679.0
  }
}
```



Observation: 0.1787937523347393
Thought: The YoY growth rate is approximately 17.88%.

Action:
```json
{
  "action": "Final Answer",
  "action_input": "The YoY growth rate of the 不動產、廠房及設備 financial statement item for ticker 3665 from year 111 to year 112 in season 4 is approximately 17.88%."
}
```

> Finished chain.
The YoY growth rate of the 不動產、廠房及設備 financial statement item for ticker 3665 from year 111 to year 112 in season 4 is approximately 17.88%.


## Explain the Growth in Two Items

In [13]:
explain_template = """
The financial statement information is as follows:
{item1}{year1}至{year2}年之年成長率: {growth1}
{item2}{year1}至{year2}年之年成長率: {growth2}

請對此兩個會計項目的年成長率作出詳細解釋。根據這兩個指標的消長，此公司的營運方針有何優缺點?
"""

explain_prompt = ChatPromptTemplate.from_messages([
    ("user", explain_template),
])

explain_chain = explain_prompt | llm
explain_result = explain_chain.invoke({"year1": 111, "year2": 112, 
                                       "item1": "不動產、廠房及設備", "item2": "營業收入",
                                       "growth1": ppe_result["output"], "growth2": sales_result["output"]})

print(explain_result)


Translation:
The growth rates of the following financial statement items for ticker 3665 from year 111 to year 112 in season 4 are as follows:
Real Estate, Factories, and Equipment: Approximately 17.88% YoY growth rate.
Revenue: Approximately -5.03% YoY growth rate.

Detailed explanation for the growth rates of these two accounting items:

The first item, "Real Estate, Factories, and Equipment," refers to the company's property, plants, and equipment. The 17.88% YoY growth rate indicates that the value of these assets has increased significantly over the year. This could be due to various reasons such as the acquisition of new assets, the appreciation of existing assets, or the revaluation of assets. This growth is a positive sign, as it suggests that the company is investing in its long-term growth and that its assets are gaining value.

The second item, "Revenue," refers to the total income that the company has generated during the year. The -5.03% YoY growth rate indicates that the

## Gradio Interface

Due to the randomness of the agents, errors might take place. Submitting for the second time may solve the issues.

In [14]:
import gradio as gr

In [15]:
def demo_func(ticker, year):
    year2 = int(year)
    year1 = year2 - 1
    
    sales_result = agent_executor.invoke({"financial_statement_item": "營業收入",
                                          "ticker": ticker, "year1": year1, "year2": year2})
    ppe_result = agent_executor.invoke({"financial_statement_item": "不動產、廠房及設備",
                                          "ticker": ticker, "year1": year1, "year2": year2})
    eps_result = agent_executor.invoke({"financial_statement_item": "稀釋每股盈餘",
                                          "ticker": ticker, "year1": year1, "year2": year2})
    cash_result = agent_executor.invoke({"financial_statement_item": "現金",
                                          "ticker": ticker, "year1": year1, "year2": year2})
    ar_result = agent_executor.invoke({"financial_statement_item": "應收帳款淨額",
                                          "ticker": ticker, "year1": year1, "year2": year2})
    
    
    sales_ppe_explain = explain_chain.invoke({"year1": year1, "year2": year2, 
                                              "item1": "不動產、廠房及設備", "item2": "營業收入",
                                               "growth1": ppe_result["output"], "growth2": sales_result["output"]})
    sales_eps_explain = explain_chain.invoke({"year1": year1, "year2": year2, 
                                              "item1": "營業收入", "item2": "EPS",
                                               "growth1": sales_result["output"], "growth2": eps_result["output"]})
    cash_ar_explain = explain_chain.invoke({"year1": year1, "year2": year2, 
                                              "item1": "現金", "item2": "應收帳款",
                                               "growth1": cash_result["output"], "growth2": ar_result["output"]})
    
    
    return sales_ppe_explain, sales_eps_explain, cash_ar_explain

In [16]:
demo = gr.Interface(
    demo_func,
    inputs=[
        gr.Textbox(label="Ticker"),
        gr.Textbox(label="Year")
    ],
    outputs=[
        gr.Textbox(label="Sales and PP&E Analysis"),
        gr.Textbox(label="Sales and EPS Analysis"),
        gr.Textbox(label="Cash and Accounts Receivable Analysis")
    ]
)

demo.launch(debug=True, share=True)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://31b1de8c166eeddfeb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
Thought: The human is asking for the YoY growth rate of the income statement item "營業收入" (Revenue) for ticker 2330 from year 111 to year 112 in season 4. I will first try to find the income statement item for the given ticker, year, and season. If not found, I will try to find it in the balanced sheet.

Action:
```
{
  "action": "GetIncomeStatementItem",
  "action_input": {
    "income_statement_item": "營業收入",
    "ticker": 2330,
    "year": 111,
    "season": 4
  }
}
```


民國111年第4季              
                                 單位：新台幣仟元              
                                     會計項目         111年度
                       Unnamed: 0_level_3            金額
0                                  營業收入合計  2.263891e+09
1                                  營業成本合計  9.155365e+08
2                                營業毛利（毛損）  1.348355e+09
3                              營業毛利（毛損）淨額  1.348355e+09
5                                    推銷費用  9.920446e+06
6                                    管理費用  5.352490e+07
7                                  研究發展費用  1.632622e+08
8                                  營業費用合計  2.267076e+08
10                              其他收益及費損淨額 -3.684030e+05
11                               營業利益（損失）  1.121279e+09
13                                   利息收入  2.242221e+07
14                                   其他收入  9.476970e+05
15                              其他利益及損失淨額  3.493586e+06
16                                 財務成本淨額  1.174998e+07
17                 採用權益法認列之關聯企業及合資損益之份額淨額  7.798359e+06
18                             營業外收入及支出合計  2.291187e+07
19                               稅前淨利（淨損）  1.144191e+09
20                            所得稅費用（利益）合計  1.272902e+08
21                         繼續營業單位本期淨利（淨損）  1.016901e+09
22                               本期淨利（淨損）  1.016901e+09
24                            確定福利計畫之再衡量數 -8.230600e+05
25          透過其他綜合損益按公允價值衡量之權益工具投資未實現評價損益 -2.637490e+05
26                        避險工具之損益－不重分類至損益  0.000000e+00
27   採用權益法認列之關聯企業及合資之其他綜合損益之份額-不重分類至損益之項目  1.544570e+05
28                         與不重分類之項目相關之所得稅 -7.339560e+05
29                            不重分類至損益之項目： -1.983960e+05
30                      國外營運機構財務報表換算之兌換差額  5.084561e+07
31          透過其他綜合損益按公允價值衡量之債務工具投資未實現評價損益 -1.010266e+07
32                                避險工具之損益  1.329231e+06
33  採用權益法認列之關聯企業及合資之其他綜合損益之份額-可能重分類至損益之項目  5.503380e+05
34                        與可能重分類之項目相關之所得稅 -6.036000e+03
35                         後續可能重分類至損益之項目：  4.262856e+07
36                             其他綜合損益（淨額）  4.243016e+07
37                               本期綜合損益總額  1.059331e+09
39                            母公司業主（淨利∕損）  1.016530e+09
40                            非控制權益（淨利∕損）  3.702660e+05
41                            母公司業主（綜合損益）  1.059125e+09
42                            非控制權益（綜合損益）  2.057900e+05
44                                 基本每股盈餘  3.920000e+01
46                                 稀釋每股盈餘  3.920000e+01


Observation: 2263891292.0
Thought: I found the income statement item for the given ticker, year, and season. Now I will find the income statement item for the previous year.

Action:
```
{
  "action": "GetIncomeStatementItem",
  "action_input": {
    "income_statement_item": "營業收入",
    "ticker": 2330,
    "year": 110,
    "season": 4
  }
}
```



民國110年第4季              
                                 單位：新台幣仟元              
                                     會計項目         110年度
                       Unnamed: 0_level_3            金額
0                                  營業收入合計  1.587415e+09
1                                  營業成本合計  7.678778e+08
2                                營業毛利（毛損）  8.195373e+08
3                              營業毛利（毛損）淨額  8.195373e+08
5                                    推銷費用  7.558591e+06
6                                    管理費用  3.692959e+07
7                                  研究發展費用  1.247348e+08
8                                  營業費用合計  1.692229e+08
10                              其他收益及費損淨額 -3.334350e+05
11                               營業利益（損失）  6.499809e+08
13                                   利息收入  5.708765e+06
14                                   其他收入  9.731410e+05
15                              其他利益及損失淨額  6.274645e+06
16                                 財務成本淨額  5.414218e+06
17                 採用權益法認列之關聯企業及合資損益之份額淨額  5.603084e+06
18                             營業外收入及支出合計  1.314542e+07
19                               稅前淨利（淨損）  6.631263e+08
20                            所得稅費用（利益）合計  6.605318e+07
21                         繼續營業單位本期淨利（淨損）  5.970731e+08
22                               本期淨利（淨損）  5.970731e+08
24                            確定福利計畫之再衡量數  2.420790e+05
25          透過其他綜合損益按公允價值衡量之權益工具投資未實現評價損益  1.900797e+06
26                        避險工具之損益－不重分類至損益 -4.141600e+04
27   採用權益法認列之關聯企業及合資之其他綜合損益之份額-不重分類至損益之項目 -3.019400e+04
28                         與不重分類之項目相關之所得稅  8.526900e+04
29                            不重分類至損益之項目：  1.985997e+06
30                      國外營運機構財務報表換算之兌換差額 -6.181830e+06
31          透過其他綜合損益按公允價值衡量之債務工具投資未實現評價損益 -3.431791e+06
32                                避險工具之損益  1.315350e+05
33  採用權益法認列之關聯企業及合資之其他綜合損益之份額-可能重分類至損益之項目 -1.199970e+05
34                        與可能重分類之項目相關之所得稅  3.370000e+03
35                         後續可能重分類至損益之項目： -9.605453e+06
36                             其他綜合損益（淨額） -7.619456e+06
37                               本期綜合損益總額  5.894537e+08
39                            母公司業主（淨利∕損）  5.965400e+08
40                            非控制權益（淨利∕損）  5.331210e+05
41                            母公司業主（綜合損益）  5.889181e+08
42                            非控制權益（綜合損益）  5.356190e+05
44                                 基本每股盈餘  2.301000e+01
46                                 稀釋每股盈餘  2.301000e+01

...


Observation: -1
Thought: The income statement item was not found. I will try to find it in the balanced sheet.

Action:
```json
{
  "action": "GetBalancedSheetItem",
  "action_input": {
    "balanced_sheet_item": "應收帳款淨額",
    "ticker": 2330,
    "year": 111,
    "season": 4
  }
}
```



民國111年第4季              
                       單位：新台幣仟元              
                           會計項目    111年12月31日
             Unnamed: 0_level_3            金額
1                       現金及約當現金  1.342814e+09
2           透過損益按公允價值衡量之金融資產－流動  1.070398e+06
3       透過其他綜合損益按公允價值衡量之金融資產－流動  1.229985e+08
4              按攤銷後成本衡量之金融資產－流動  9.460022e+07
5                    避險之金融資產－流動  2.329000e+03
6                        應收帳款淨額  2.297559e+08
7                    應收帳款－關係人淨額  1.583958e+06
8                   其他應收款－關係人淨額  6.897500e+04
9                            存貨  2.211491e+08
10                       其他流動資產  3.885320e+07
11                       流動資產合計  2.052897e+09
13     透過其他綜合損益按公允價值衡量之金融資產－非流動  6.159200e+06
14            按攤銷後成本衡量之金融資產－非流動  3.512722e+07
15                     採用權益法之投資  2.764150e+07
16                    不動產、廠房及設備  2.693837e+09
17                        使用權資產  4.191414e+07
18                         無形資產  2.599916e+07
19                      遞延所得稅資產  6.918584e+07
20                      其他非流動資產  1.201811e+07
21                      非流動資產合計  2.911882e+09
22                         資產總額  4.964779e+09
24                         短期借款  0.000000e+00
25          透過損益按公允價值衡量之金融負債－流動  1.162150e+05
26                   避險之金融負債－流動  8.130000e+02
27                         應付帳款  5.487971e+07
28                     應付帳款－關係人  1.642637e+06
29                        其他應付款  4.543008e+08
30                      本期所得稅負債  1.208018e+08
31                       其他流動負債  3.124848e+08
32                       流動負債合計  9.442268e+08
34                        應付公司債  8.343364e+08
35                         長期借款  4.760047e+06
36                      遞延所得稅負債  1.031383e+06
37                     租賃負債－非流動  2.976410e+07
38                      其他非流動負債  1.901712e+08
39                      非流動負債合計  1.060063e+09
40                         負債總額  2.004290e+09
43                        普通股股本  2.593038e+08
44                         股本合計  2.593038e+08
46                    資本公積－發行溢價  3.307602e+07
47   資本公積－實際取得或處分子公司股權價格與帳面價值差額  8.406282e+06
48          資本公積-認列對子公司所有權權益變動數  4.229892e+06
49                    資本公積－受贈資產  6.495500e+04
50  資本公積－採用權益法認列關聯企業及合資股權淨值之變動數  3.118630e+05
51                    資本公積－合併溢額  2.280329e+07
52                資本公積－限制員工權利股票  4.380290e+05
53                       資本公積合計  6.933033e+07
55                       法定盈餘公積  3.111469e+08
56                       特別盈餘公積  3.154310e+06
57                未分配盈餘（或待彌補虧損）  2.323223e+09
58                       保留盈餘合計  2.637525e+09
60                       其他權益合計 -2.050563e+07
61                         庫藏股票  0.000000e+00
62                歸屬於母公司業主之權益合計  2.945653e+09
63                        非控制權益  1.483567e+07
64                         權益總額  2.960489e+09
65                      負債及權益總計  4.964779e+09
66      預收股款（權益項下）之約當發行股數（單位：股）  0.000000e+00
67    母公司暨子公司所持有之母公司庫藏股股數（單位：股）  0.000000e+00


Observation: 229755887.0
Thought: I have found the value of the financial statement item '應收帳款淨額' for ticker '2330' in year '111' and season '4'. Now, I will find its value in year '112'.

Action:
```json
{
  "action": "GetBalancedSheetItem",
  "action_input": {
    "balanced_sheet_item": "應收帳款淨額",
    "ticker": 2330,
    "year": 112,
    "season": 4
  }
}
```



民國112年第4季              
                       單位：新台幣仟元              
                           會計項目    112年12月31日
             Unnamed: 0_level_3            金額
1                       現金及約當現金  1.465428e+09
2           透過損益按公允價值衡量之金融資產－流動  9.246360e+05
3       透過其他綜合損益按公允價值衡量之金融資產－流動  1.545308e+08
4              按攤銷後成本衡量之金融資產－流動  6.676122e+07
5                    避險之金融資產－流動  0.000000e+00
6                        應收帳款淨額  2.013139e+08
7                    應收帳款－關係人淨額  6.244510e+05
8                   其他應收款－關係人淨額  7.187100e+04
9                            存貨  2.509971e+08
10                       其他流動資產  5.338115e+07
11                       流動資產合計  2.194033e+09
13         透過損益按公允價值衡量之金融資產－非流動  1.341746e+07
14     透過其他綜合損益按公允價值衡量之金融資產－非流動  7.208655e+06
15            按攤銷後成本衡量之金融資產－非流動  7.919937e+07
16                     採用權益法之投資  2.961664e+07
17                    不動產、廠房及設備  3.064475e+09
18                        使用權資產  4.042483e+07
19                         無形資產  2.276674e+07
20                      遞延所得稅資產  6.417579e+07
21                      其他非流動資產  1.705384e+07
22                      非流動資產合計  3.338338e+09
23                         資產總額  5.532371e+09
25          透過損益按公允價值衡量之金融負債－流動  1.214120e+05
26                   避險之金融負債－流動  2.733416e+07
27                         應付帳款  5.572676e+07
28                     應付帳款－關係人  1.566300e+06
29                        其他應付款  4.239606e+08
30                      本期所得稅負債  9.891290e+07
31                       其他流動負債  3.059612e+08
32                       流動負債合計  9.135833e+08
34                        應付公司債  9.138998e+08
35                         長期借款  4.382965e+06
36                      遞延所得稅負債  5.385600e+04
37                     租賃負債－非流動  2.868184e+07
38                      其他非流動負債  1.885066e+08
39                      非流動負債合計  1.135525e+09
40                         負債總額  2.049108e+09
43                        普通股股本  2.593207e+08
44                         股本合計  2.593207e+08
46                    資本公積－發行溢價  3.329922e+07
47   資本公積－實際取得或處分子公司股權價格與帳面價值差額  8.406282e+06
48          資本公積-認列對子公司所有權權益變動數  4.199936e+06
49                    資本公積－受贈資產  8.136800e+04
50  資本公積－採用權益法認列關聯企業及合資股權淨值之變動數  3.023960e+05
51                    資本公積－合併溢額  2.280329e+07
52                資本公積－限制員工權利股票  7.838830e+05
53                       資本公積合計  6.987638e+07
55                       法定盈餘公積  3.111469e+08
56                       特別盈餘公積  0.000000e+00
57                未分配盈餘（或待彌補虧損）  2.846884e+09
58                       保留盈餘合計  3.158031e+09
60                       其他權益合計 -2.831426e+07
61                         庫藏股票  0.000000e+00
62                歸屬於母公司業主之權益合計  3.458914e+09
63                        非控制權益  2.434922e+07
64                         權益總額  3.483263e+09
65                      負債及權益總計  5.532371e+09
66      預收股款（權益項下）之約當發行股數（單位：股）  0.000000e+00
67    母公司暨子公司所持有之母公司庫藏股股數（單位：股）  0.000000e+00


Observation: 201313914.0
Thought: I have found the value of the financial statement item '應收帳款淨額' for ticker '2330' in year '112' and season '4'. Now, I will calculate the YoY growth rate.

Action:
```json
{
  "action": "GetYoYGrowth",
  "action_input": {
    "previous_year_value": 229755887.0,
    "current_year_value": 201313914.0
  }
}
```


Observation: -0.12379214030759525
Thought: The YoY growth rate of the financial statement item '應收帳款淨額' for ticker '2330' from year '111' to year '112' in season '4' is approximately -12.38%.

Action:
```json
{
  "action": "Final Answer",
  "action_input": "The YoY growth rate of the financial statement item '應收帳款淨額' for ticker '2330' from year 111 to year 112 in season 4 is approximately -12.38%."
}
```

> Finished chain.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://31b1de8c166eeddfeb.gradio.live
